In [19]:
# Import required libraries
import rasterio as rio
import numpy as np

import os
import re

### Importing the Image
We begin by importing the image using rasterio.

In [2]:
image_file = r"C:\Users\RQ\Documents\PCA\GC_Landsat8_SR.tif"
sat_data = rio.open(image_file)

### Bands
Let's check how many bands our image has.

In [3]:
print('Bands: {}'.format(sat_data.count))

# Sequence of band indexes
print(sat_data.indexes)

Bands: 12
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)


As we can see, our bands are not 0-indexed and we have 12 bands to work with.

### Data Preprocessing

In [4]:
src_meta = sat_data.meta

src_meta

{'driver': 'GTiff',
 'dtype': 'float64',
 'nodata': None,
 'width': 635,
 'height': 1255,
 'count': 12,
 'crs': CRS.from_dict(init='epsg:4326'),
 'transform': Affine(0.00013474729261792824, 0.0, -74.7413587747272,
        0.0, -0.00013474729261792824, 7.166669505177131)}

In [5]:
n_bands = src_meta['count']
height = src_meta['height']
width = src_meta['width']

n_bands, height, width

(12, 1255, 635)

In [6]:
# Read the image with all bands
img = sat_data.read()

img

array([[[  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [310. , 310. , 235. , ..., 229. , 274.5, 274.5],
        [310. , 310. , 235. , ..., 243. , 285. , 285. ],
        ...,
        [230. , 237. , 237. , ..., 148. , 148. , 153.5],
        [230. , 237. , 237. , ..., 148. , 148. , 153.5],
        [230. , 306. , 306. , ..., 143.5, 143.5, 167.5]],

       [[  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [360. , 360. , 245. , ..., 255.5, 298. , 298. ],
        [360. , 360. , 245. , ..., 264. , 296. , 296. ],
        ...,
        [314. , 308. , 308. , ..., 215.5, 215.5, 232.5],
        [314. , 308. , 308. , ..., 215.5, 215.5, 232.5],
        [310. , 396. , 396. , ..., 236.5, 236.5, 284. ]],

       [[  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [652. , 652. , 526. , ..., 457. , 558.5, 558.5],
        [652. , 652. , 526. , ..., 467.5, 553. , 553. ],
        ...,
        [712. , 706.5, 706.5, ..., 595. , 595. , 628.5],
        [712. , 706.5, 706.5, ..., 595. , 595

In [7]:
# shape of our image
img.shape

(12, 1255, 635)

In [8]:
# Access just the second band 
b2 = sat_data.read(2)

b2.shape

(1255, 635)

In [9]:
# Let's access the third band too 
b3 = sat_data.read(3)

b3.shape

(1255, 635)

### Transforming Image with Ratio
We want to try using the equation "(b3-b2)/(b3+2)" as our first example of the equation

In [10]:
equation = '(b3-b2)/(b3+b2)'

In [11]:
# Our result of using this equation with our predefined band 2 and 3 
result = eval(equation)

result.shape

(1255, 635)

In [12]:
result

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [0.28853755, 0.28853755, 0.36446174, ..., 0.28280702, 0.30414478,
        0.30414478],
       [0.28853755, 0.28853755, 0.36446174, ..., 0.27819549, 0.30270907,
        0.30270907],
       ...,
       [0.38791423, 0.39280434, 0.39280434, ..., 0.46822949, 0.46822949,
        0.45993031],
       [0.38791423, 0.39280434, 0.39280434, ..., 0.46822949, 0.46822949,
        0.45993031],
       [0.42083139, 0.37637795, 0.37637795, ..., 0.49112426, 0.49112426,
        0.47285383]])

As we can see, result has the shape of a single-band image with our ratio in the equation.

After we have our image transformed based on the image, we want to be able to reshape it to (1, height, width) and then export it.

In [13]:
result = result.reshape(1, height, width)

result

array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [0.28853755, 0.28853755, 0.36446174, ..., 0.28280702,
         0.30414478, 0.30414478],
        [0.28853755, 0.28853755, 0.36446174, ..., 0.27819549,
         0.30270907, 0.30270907],
        ...,
        [0.38791423, 0.39280434, 0.39280434, ..., 0.46822949,
         0.46822949, 0.45993031],
        [0.38791423, 0.39280434, 0.39280434, ..., 0.46822949,
         0.46822949, 0.45993031],
        [0.42083139, 0.37637795, 0.37637795, ..., 0.49112426,
         0.49112426, 0.47285383]]])

In [14]:
# Let's confirm our shape
result.shape

(1, 1255, 635)

### Export the Image
Now that we have our transfomred image in the format that we want, let's export it back into the original filepath with a new name.

In [15]:
# Set the destination filepath
dst_fp = r"C:\Users\RQ\Documents\PCA\GC_Landsat8_ratio.tif"

# Create metadata for the destination file
dst_meta = src_meta.copy()

dst_meta['count'] = 1 

dst_meta

{'driver': 'GTiff',
 'dtype': 'float64',
 'nodata': None,
 'width': 635,
 'height': 1255,
 'count': 1,
 'crs': CRS.from_dict(init='epsg:4326'),
 'transform': Affine(0.00013474729261792824, 0.0, -74.7413587747272,
        0.0, -0.00013474729261792824, 7.166669505177131)}

In [16]:
# Open a new file in 'write' mode and unpack (**) the destination metadata
with rio.open(dst_fp, 'w', **dst_meta) as dst:
    dst.write(result)

## Generalizing Function
We want to be able to input a string that is an equation, read only the required bands from our inputted image and then perform the equation on the bands and export the new image.

In [17]:
# Let's use the same equation
equation

'(b3-b2)/(b3+b2)'

In [20]:
# Let's get all the bands in the equation
band_labels = re.findall("[Bb]"+str(list(sat_data.indexes)), equation)
band_labels = list(dict.fromkeys(band_labels))

band_labels

['b3', 'b2']

In [21]:
list(sat_data.indexes)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [22]:
# Create a dictionary with all the bands
bands = {'B%d'%(i+1): img[i] for i in list(range(img.shape[0]))}

bands

{'B1': array([[  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [310. , 310. , 235. , ..., 229. , 274.5, 274.5],
        [310. , 310. , 235. , ..., 243. , 285. , 285. ],
        ...,
        [230. , 237. , 237. , ..., 148. , 148. , 153.5],
        [230. , 237. , 237. , ..., 148. , 148. , 153.5],
        [230. , 306. , 306. , ..., 143.5, 143.5, 167.5]]),
 'B2': array([[  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [360. , 360. , 245. , ..., 255.5, 298. , 298. ],
        [360. , 360. , 245. , ..., 264. , 296. , 296. ],
        ...,
        [314. , 308. , 308. , ..., 215.5, 215.5, 232.5],
        [314. , 308. , 308. , ..., 215.5, 215.5, 232.5],
        [310. , 396. , 396. , ..., 236.5, 236.5, 284. ]]),
 'B3': array([[  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [652. , 652. , 526. , ..., 457. , 558.5, 558.5],
        [652. , 652. , 526. , ..., 467.5, 553. , 553. ],
        ...,
        [712. , 706.5, 706.5, ..., 595. , 595. , 628.5],
        [712. , 706.5, 706.

We have the bands that are mentioned in the equations and we now have a dictionary too with all of the bands as 'B1', 'B2', ... and we have the bands mentioned in the equation for extra information. 

Now we need to be able to access these bands and their values from the dictionary when evaluating our dictionary, so we unpack the dictionary as local variables to use with eval(equation)

In [23]:
# Unpack the dictionary as local variables
locals().update(bands)

In [24]:
# Test out our variables
B1

array([[  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [310. , 310. , 235. , ..., 229. , 274.5, 274.5],
       [310. , 310. , 235. , ..., 243. , 285. , 285. ],
       ...,
       [230. , 237. , 237. , ..., 148. , 148. , 153.5],
       [230. , 237. , 237. , ..., 148. , 148. , 153.5],
       [230. , 306. , 306. , ..., 143.5, 143.5, 167.5]])

In [25]:
# Call eval(equation)
result = eval(equation)

result.shape

(1255, 635)

In [26]:
# Reshape image 
result = result.reshape(1, height, width)

result.shape

(1, 1255, 635)

The next step would be to export the image, but in this case we have shown how to do that above so we won't follow through with that step.


## Multi-Equation Transformation
Now what if we wanted to try this with more than one equation and create a multi-band image. The following below will create a 3-band image with the equations: 'B6/B7', 'B6/B4', 'B4/B2' which shows a color composite for RGB.

In [27]:
# Let's create our equations in string form
eqn1 = 'B6/B7'
eqn2 = 'B6/B4'
eqn3 = 'B4/B2'

In [72]:
# Create a resulting transformation for each equation
result1 = eval(eqn1)
result2 = eval(eqn2)
result3 = eval(eqn3)

results = [result1, result2, result3]

result1.shape, result2.shape, result3.shape

((1255, 635), (1255, 635), (1255, 635))

We need to append these all into a (len(equations), height, width) array.

In [67]:
# We'd get the num_eqns from the *args in the function, but in this case we set the variable to 3
num_eqns = 3 

# Create an array (num_eqns, height, width) with zeros
transformed_array = np.zeros((num_eqns, height, width))

transformed_array.shape

(3, 1255, 635)

In [73]:
# Each equation result will go into the array in the respective order
for i in range(num_eqns):
    transformed_array[i] = results[i]

transformed_array

array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [2.18918919, 2.18918919, 2.34668508, ..., 2.66923736,
         2.5712582 , 2.5712582 ],
        [2.18918919, 2.18918919, 2.34668508, ..., 2.6190901 ,
         2.46415553, 2.46415553],
        ...,
        [1.95159386, 1.96782077, 1.96782077, ..., 2.39855072,
         2.39855072, 2.29842932],
        [1.95159386, 1.96782077, 1.96782077, ..., 2.39855072,
         2.39855072, 2.29842932],
        [2.08189482, 1.92459239, 1.92459239, ..., 2.35681369,
         2.35681369, 2.17510854]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [3.77068966, 3.77068966, 4.73259053, ..., 5.73664825,
         5.97229917, 5.97229917],
        [3.77068966, 3.77068966, 4.73259053, ..., 5.16901408,
         5.31585845, 5.31585845],
        ...,
        [3.73418675, 3.59985097, 3.59985097, ..., 5.56302521,
         5.56302521, 5.06538462],
        [3.7

In [74]:
# Check shape again
transformed_array.shape

(3, 1255, 635)

## Creating general functions

In [88]:
def transform_image(input_fp, output_fp=None, *equations):
    ''' 
    Transforms an image of format .TIF from the input_fp using the equation 
    given and then exports it to output_fp. 
    
    Args: 
        input_fp (str): The file path to retrieve the .TIF file from 
        output_fp (str): The file path to export the image to once it is 
        transformed.
            None, _transformed will be added to the input filepath (default: None)
        *equations: Variable length argument list of equations to transform the image; image will
        be transformed in this specific order. Must refer to bands as B1, B2, B3, etc. 
        
    Returns:
        None, it will print that the New Transformed Image is stored in the 
        output_fp
        
    @author: Kanika Chopra
    '''
    # Import and open the image
    sat_data = rio.open(input_fp)
    
    # Gather the metadata and collect height and width
    src_meta = sat_data.meta
    height = src_meta['height']
    width = src_meta['width']

    # Read the image
    img = sat_data.read()
    
    # Create a dictionary with all the bands
    bands = {'B%d'%(i+1): img[i] for i in list(range(img.shape[0]))}
    
    # Unpack the dictionary as local variables
    locals().update(bands)
    
    # Evaluate our equations on the bands and append to an overall array
    num_eqns = len(equations)
    
    # Create an empty array to index your results into later
    transformed_array = np.zeros((num_eqns, height, width))
    
    for i in range(num_eqns):
        transformed_array[i] = eval(equations[i])
        print('Equation %d:' % (i+1), equations[i])
    
    # Export the image 
    if output_fp==None:
        output_fp = append_file_suffix(input_fp, 'transformed')
        
    # Recreate the metadata and set count to 1 since we want a single-band image
    dst_meta = src_meta.copy()
    dst_meta['count'] = 1 
    
    with rio.open(output_fp, 'w', **dst_meta) as dst:
        dst.write(result)
    
    print('New Image after equation(s) applied stored in ' + output_fp)
    
def append_file_suffix(filepath, suffix=None):
    ''' Appends a suffix to a filepath. 
    Args: 
        filepath (str): The file path to modify. 
        suffix (str): The suffix to be appended to the filepath string 
            None, no suffix is appended (default: None)
    Returns: 
        str: The file path with added suffix
    @author: charles
    '''
    name, ext = os.path.splitext(filepath)
    if suffix is not None: 
        filepath = "{name}_{uid}{ext}".format(name=name, uid=suffix, ext=ext)
        
    return filepath

In [89]:
input_fp = r"C:\Users\RQ\Documents\PCA\GC_Landsat8_SR.tif"
test_equation = '(B3-B2)/(B3+B2)'
transform_image(input_fp, None, eqn1, eqn2, eqn3)

Equation 1: B6/B7
Equation 2: B6/B4
Equation 3: B4/B2
New Image after equation(s) applied stored in C:\Users\RQ\Documents\PCA\GC_Landsat8_SR_transformed.tif
